# Chapter 10: Special Cases of Pole Placement Control

## Example 10.10: Design_of_conventional_controller_for_van_de_Vusse_reactor_problem.sce

In [ ]:
// Design of conventional controller for van de Vusse reactor problem
// 10.10

exec('tf.sci');
exec('myc2d.sci');
exec('imcsplit.sci',-1);
exec('imc_stable.sci');
exec('zpowk.sci',-1);
exec('flip.sci',-1);
exec('poladd.sci',-1);
exec('polsize.sci',-1);

num = [-1.117 3.1472]; den = [1 4.6429 5.3821];
G = tf(num,den);
Ts = 0.1;
[B,A,k] = myc2d(G,Ts);
alpha = 0.5;
[k,HiN,HiD,R,S] = imc_stable(B,A,k,alpha);
[zk,dzk] = zpowk(k);
Bp = B; Ap = A;






## Example 10.1: Effect_of_delay_in_control_performance.sce

In [ ]:
// Effect of delay in control performance
// 10.1 

exec('zpowk.sci',-1);
exec('pp_im.sci',-1);
exec('cosfil_ip.sci',-1);
exec('polsplit3.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polyno.sci',-1);

Ts = 1; B = 0.63; A = [1 -0.37];
k = input('Enter the delay as an integer: ');
if k<=0, k = 1; end
[zk,dzk] = zpowk(k);

// Desired transfer function
phi = [1 -0.5];
delta = 1; // internal model of step introduced

// Controller design
[Rc,Sc,Tc,gamm] = pp_im(B,A,k,phi,delta);

// simulation parameters for stb_disc.xcos 
// y1: 0 to 1; u1: 0 to 1.2
st = 1.0; // desired change in setpoint
t_init = 0; // simulation start time
t_final = 20; // simulation end time

// simulation parameters for stb_disc.xcos
N_var = 0; C = 0; D = 1; N = 1; 

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Rcp1,Rcp2] = cosfil_ip(1,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Bp,Ap] = cosfil_ip(B,A); // B/A
[zkp1,zkp2] = cosfil_ip(zk,1); // zk/1
[Cp,Dp] = cosfil_ip(C,D); // C/D


## Example 10.2: Smith_predictor_for_paper_machine_control.sce

In [ ]:
// Smith predictor for paper machine control in Example 10.2 on page 385.
// 10.2

exec('zpowk.sci',-1);
exec('poladd.sci',-1);
exec('polsize.sci',-1);
exec('pp_im.sci',-1);
exec('polsplit3.sci',-1);
exec('polmul.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('seshft.sci',-1);
exec('cosfil_ip.sci',-1);
exec('polyno.sci',-1);

Ts = 1; B = 0.63; A = [1 -0.37]; k = 3;
Bd = convol(B,[0 1]);
kd = k - 1;
[zkd,dzkd] = zpowk(kd);
[mzkd,dmzkd] = poladd(1,0,-zkd,dzkd);

// Desired transfer function
phi = [1 -0.5]; delta = 1;

// Controller design
[Rc,Sc,Tc,gamm] = pp_im(B,A,1,phi,delta);

// simulation parameters for smith_disc.xcos 
st = 1.0; // desired change in setpoint
t_init = 0; // simulation start time
t_final = 20; // simulation end time

// simulation parameters for smith_disc.xcos
N_var = 0; C = 0; D = 1; N = 1; 

[Tcp1,Tcp2] = cosfil_ip(Tc,1); // Tc/1
[Rcp1,Rcp2] = cosfil_ip(1,Rc); // 1/Rc
[Scp1,Scp2] = cosfil_ip(Sc,1); // Sc/1
[Bdp,Ap] = cosfil_ip(Bd,A); // Bd/Ad
[zkdp1,zkdp2] = cosfil_ip(zkd,1); // zkd/1
[mzkdp1,mzkdp2] = cosfil_ip(mzkd,1); // mzkd/1
[Cp,Dp] = cosfil_ip(C,D); // C/D


## Example 10.3: Splitting_a_polynomial_B.sci

In [ ]:
// Splitting a polynomial B(z)
// 10.3
// Splits a polynomial B into good, nonminimum with 
// positive real & with negative real parts.
// All are returned in polynomial form.
// Gain is returned in Kp and delay in k.  

function [Kp,k,Bg,Bnmp,Bm] = imcsplit(B,polynomial)
k = 0;
Kp = 1;
if(polynomial)
  rts = roots(B);
  Kp = sum(B)/sum(coeff(poly(rts,'z')));
else
  rts = B;
end
Bg = 1; Bnmp = 1; Bm = 1;
for i = 1:length(rts),
    rt = rts(i);
    if rt == 0,
       k = k+1;
    elseif (abs(rt)<1 & real(rt)>=0)
      Bg = convol(Bg,[1 -rt]);
    elseif (abs(rt)>=1 & real(rt)>=0)
      Bnmp = convol(Bnmp,[1 -rt]);
    else
      Bm = convol(Bm,[1 -rt]);
    end
end









## Example 10.4: Design_of_internal_model_controller.sci

In [ ]:
// Design of internal model controller
// 10.4
// Designs Discrete Internal Model Controller
// for transfer function z^{-k}B(z^{-1})/A(z^{-1})
// Numerator and Denominator of IMC HQ are outputs
// Controller is also given in R,S form 

function [k,HiN,HiD] = imc_stable1(B,A,k,alpha)

[Kp,d,Bg,Bnmp,Bm] = imcsplit(B,mtlb_logical(1));
Bg = Kp * Bg;
Bnmpr = flip(Bnmp);
Bms = sum(Bm);
HiN = A;
HiD = Bms * convol(Bg,Bnmpr);
k = k+d;
endfunction;



## Example 10.5: Flipping_a_vector.sci

In [ ]:
// 10.5
function b = flip(a)
b = a(length(a):-1:1);
endfunction;

## Example 10.6: IMC_design_for_viscosity_control_problem.sce

In [ ]:
// IMC design for viscosity control problem
// 10.6

exec('imc_stable1.sci',-1);
exec('zpowk.sci',-1);
exec('imcsplit.sci',-1);
exec('flip.sci',-1);

B = [0.51 1.21];
A = [1 -0.44];
k = 1;
alpha = 0.5;

[k,GiN,GiD] = imc_stable1(B,A,k,alpha);

[zk,dzk] = zpowk(k);
Bp = B; Ap = A; 
Ts = 0.1; t0 = 0; tf = 20; Nvar = 0.01;



## Example 10.7: IMC_design_for_the_control_of_van_de_Vusse_reactor.sce

In [ ]:
// IMC design for the control of van de Vusse reactor
// 10.7

exec('tf.sci');
exec('myc2d.sci');
exec('imc_stable1.sci');
exec('imcsplit.sci',-1);
exec('flip.sci',-1);
exec('zpowk.sci',-1);

num = [-1.117 3.1472]; den = [1 4.6429 5.3821];
G = tf(num,den);
Ts = 0.1;
[B,A,k] = myc2d(G,Ts);
alpha = 0.9;
[k,GiN,GiD] = imc_stable1(B,A,k,alpha);
[zk,dzk] = zpowk(k);
Bp = B; Ap = A;
t0 = 0; tfi = 10; st = 1; Nvar = 0;



## Example 10.8: IMC_design_for_an_example_by_Lewin.sce

In [ ]:
// IMC design for Lewin's example
// 10.8

exec('tf.sci');
exec('myc2d.sci');
exec('imc_stable1.sci');
exec('zpowk.sci',-1);
exec('imcsplit.sci',-1);
exec('flip.sci',-1);

num = 1; den = [250 35 1]; Ts = 3;
G = tf(num,den);

[B,A,k] = myc2d(G,Ts);

alpha = 0.9;
[k,GiN,GiD] = imc_stable1(B,A,k,alpha);

[zk,dzk] = zpowk(k);
Bp = B; Ap = A;
t0 = 0; tfi = 100; st = 1; Nvar = 0;




## Example 10.9: EX10_9.sci

In [ ]:
// Design of conventional controller which is an equivalent of internal model controller
// 10.9

// Designs Discrete Internal Model Controller
// for transfer function z^{-k}B(z^{-1})/A(z^{-1})
// Numerator and Denominator of IMC HQ are outputs
// Controller is also given in R,S form 


function [k,HiN,HiD,R,S,mu] = imc_stable(B,A,k,alpha)

[Kp,d,Bg,Bnmp,Bm] = imcsplit(B,mtlb_logical(1));
Bg = Kp * Bg;

Bnmpr = flip(Bnmp);
Bms = sum(Bm);
HiN = A;
HiD = Bms * convol(Bg,Bnmpr);
k = k+d;

[zk,dzk] = zpowk(k);
Bf = (1-alpha);
Af = [1 -alpha];
S = convol(Bf,A);
R1 = convol(Af,convol(Bnmpr,Bms));
R2 = convol(zk,convol(Bf,convol(Bnmp,Bm)));

[R,dR] = poladd(R1,length(R1)-1,-R2,length(R2)-1);
R = convol(Bg,R);
endfunction;
